In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

df = pd.read_csv("loan_approval_dataset.csv")
df.head()
df.columns = df.columns.str.strip()

X = df.drop(["loan_id", "loan_status"], axis=1)
y = df["loan_status"]

le_feat = LabelEncoder()
for col in X.columns:
    if X[col].dtype == "object":
        X[col] = le_feat.fit_transform(X[col])

le_target = LabelEncoder()
y = le_target.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

print("Accuracy :", acc)
print("Precision:", prec)
print("Recall   :", rec)
print("F1-Score :", f1)
print("\nConfusion Matrix:\n", cm)
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=le_target.classes_))


Accuracy : 0.8790007806401249
Precision: 0.8185483870967742
Recall   : 0.861995753715499
F1-Score : 0.8397104446742503

Confusion Matrix:
 [[720  90]
 [ 65 406]]

Classification Report:
               precision    recall  f1-score   support

    Approved       0.92      0.89      0.90       810
    Rejected       0.82      0.86      0.84       471

    accuracy                           0.88      1281
   macro avg       0.87      0.88      0.87      1281
weighted avg       0.88      0.88      0.88      1281



In [4]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report
)

df = pd.read_csv("loan_approval_dataset.csv")
df.columns = df.columns.str.strip()

X = df.drop(["loan_id", "loan_status"], axis=1)
y = df["loan_status"]


le_feat = LabelEncoder()
for col in X.columns:
    if X[col].dtype == "object":
        X[col] = le_feat.fit_transform(X[col])

# Encode target
le_target = LabelEncoder()
y = le_target.fit_transform(y)

k = 5
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

acc_scores = []
prec_scores = []
rec_scores = []
f1_scores = []

n_classes = len(np.unique(y))
cm_total = np.zeros((n_classes, n_classes), dtype=int)

fold = 1
for train_idx, test_idx in skf.split(X, y):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

   
    knn = KNeighborsClassifier(n_neighbors=5)
    knn.fit(X_train, y_train)

    y_pred = knn.predict(X_test)

   
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    acc_scores.append(acc)
    prec_scores.append(prec)
    rec_scores.append(rec)
    f1_scores.append(f1)

    
    cm = confusion_matrix(y_test, y_pred, labels=np.arange(n_classes))
    cm_total += cm

    print(f"\n----- Fold {fold} -----")
    print("Accuracy :", acc)
    print("Precision:", prec)
    print("Recall   :", rec)
    print("F1-Score :", f1)
    fold += 1


print("\n===== Cross-Validation Results ({}-Fold) =====".format(k))
print("Mean Accuracy :", np.mean(acc_scores))
print("Mean Precision:", np.mean(prec_scores))
print("Mean Recall   :", np.mean(rec_scores))
print("Mean F1-Score :", np.mean(f1_scores))

print("\nAggregated Confusion Matrix (summed over all folds):\n", cm_total)


print("\n(Last Fold) Classification Report:\n",
      classification_report(y_test, y_pred, target_names=le_target.classes_))



----- Fold 1 -----
Accuracy : 0.9039812646370023
Precision: 0.875
Recall   : 0.8695652173913043
F1-Score : 0.8722741433021807

----- Fold 2 -----
Accuracy : 0.882903981264637
Precision: 0.8328358208955224
Recall   : 0.8637770897832817
F1-Score : 0.8480243161094225

----- Fold 3 -----
Accuracy : 0.905152224824356
Precision: 0.8853503184713376
Recall   : 0.8606811145510835
F1-Score : 0.8728414442700158

----- Fold 4 -----
Accuracy : 0.8840749414519906
Precision: 0.8478260869565217
Recall   : 0.8452012383900929
F1-Score : 0.8465116279069768

----- Fold 5 -----
Accuracy : 0.8944900351699883
Precision: 0.8580246913580247
Recall   : 0.8633540372670807
F1-Score : 0.8606811145510835

===== Cross-Validation Results (5-Fold) =====
Mean Accuracy : 0.8941204894695949
Mean Precision: 0.8598073835362813
Mean Recall   : 0.8605157394765687
Mean F1-Score : 0.8600665292279357

Aggregated Confusion Matrix (summed over all folds):
 [[2429  227]
 [ 225 1388]]

(Last Fold) Classification Report:
          

In [5]:
df

,loan_id,no_of_dependents,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value,loan_status
0,1,2,Graduate,No,9600000,29900000,12,778,2400000,17600000,22700000,8000000,Approved
1,2,0,Not Graduate,Yes,4100000,12200000,8,417,2700000,2200000,8800000,3300000,Rejected
2,3,3,Graduate,No,9100000,29700000,20,506,7100000,4500000,33300000,12800000,Rejected
3,4,3,Graduate,No,8200000,30700000,8,467,18200000,3300000,23300000,7900000,Rejected
4,5,5,Not Graduate,Yes,9800000,24200000,20,382,12400000,8200000,29400000,5000000,Rejected
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4264,4265,5,Graduate,Yes,1000000,2300000,12,317,2800000,500000,3300000,800000,Rejected
4265,4266,0,Not Graduate,Yes,3300000,11300000,20,559,4200000,2900000,11000000,1900000,Approved
4266,4267,2,Not Graduate,No,6500000,23900000,18,457,1200000,12400000,18100000,7300000,Rejected
4267,4268,1,Not Graduate,No,4100000,12800000,8,780,8200000,700000,14100000,5800000,Approved
